In [2]:
import os
import json
import zipfile
import shutil

def load_all_info(log_dir):
    per_episode_error = []
    if os.path.exists(os.path.join(log_dir, "all_info.log")):
        print(f"Loading {log_dir}all_info.log")
        with open(log_dir + "all_info.log", 'r') as fp:
            lines = fp.readlines()
            for line in lines:
                line = line.strip()
                if line != "":
                    per_episode_error.append(json.loads(line))
    else:
        raise FileNotFoundError(f"File {log_dir}all_info.log not found.")
    return per_episode_error

def save_all_info(log_dir, per_episode_error):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    with open(log_dir + "all_info.log", 'w') as fp:
        for item in per_episode_error:
            json.dump(item, fp)
            fp.write("\n")

# sample episode labels
def load_env(task_config="objectnav_hm3d_rgbd_semantic.yaml", split="val"):
    from habitat.config.default import get_config
    from habitat import Env, make_dataset
    from habitat.config import read_write

    config_env = get_config(config_path="configs/" + task_config)
    with read_write(config_env):
        config_env.habitat.dataset.split = split
    dataset = make_dataset(config_env.habitat.dataset.type, config=config_env.habitat.dataset)
    env = Env(config_env, dataset)
    return env

def get_scene_id(scene_path):
    import re
    # Get the scene_id from the path
    scene_id = scene_path.split("/")[-1]
    scene_id = re.sub(r'\.basis\.glb$', '', scene_id)
    return scene_id

def get_episode_labels(env):
    episode_labels = []
    for i in range(len(env.episodes)):
        scene_id = get_scene_id(env.episodes[i].scene_id)
        episode_id = env.episodes[i].episode_id
        episode_label = f'{scene_id}_{episode_id}'
        episode_labels.append(episode_label)
    return episode_labels

def pack_videos(output_path, output_name, videos_path1, videos_path2, episode_labels, 
               suffix1="_old", suffix2="_new", to_zip=True, to_folder=False):
    # Ensure output directory exists
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    # Track successful and failed video additions
    added_videos1 = []
    added_videos2 = []
    missing_videos1 = []
    missing_videos2 = []
    # Paths to return
    result_paths = {}
    # Create folder if requested
    folder_path = None
    if to_folder:
        folder_path = os.path.join(output_path, output_name)
        if os.path.exists(folder_path):
            # Clean the existing folder
            shutil.rmtree(folder_path)
        os.makedirs(folder_path)
        result_paths['folder'] = folder_path
    # Create the zip file if requested
    zip_file_path = None
    zip_obj = None
    if to_zip:
        zip_file_path = os.path.join(output_path, f"{output_name}.zip")
        zip_obj = zipfile.ZipFile(zip_file_path, 'w')
        result_paths['zip'] = zip_file_path
    # Process each episode
    for episode_label in episode_labels:
        # Construct video file names
        video_name = f"eps_{episode_label}_vis.mp4"
        video_path1 = os.path.join(videos_path1, video_name)
        video_path2 = os.path.join(videos_path2, video_name)
        # Get file extension
        _, ext = os.path.splitext(video_name)
        # Process video from first path
        if os.path.exists(video_path1):
            # Create new name with suffix
            new_name1 = f"eps_{episode_label}{suffix1}{ext}"
            # Add to zip if needed
            if to_zip:
                zip_obj.write(video_path1, arcname=new_name1)
            # Copy to folder if needed
            if to_folder:
                shutil.copy2(video_path1, os.path.join(folder_path, new_name1))
            added_videos1.append(episode_label)
        else:
            missing_videos1.append(episode_label)
        # Process video from second path
        if os.path.exists(video_path2):
            # Create new name with suffix
            new_name2 = f"eps_{episode_label}{suffix2}{ext}"
            # Add to zip if needed
            if to_zip:
                zip_obj.write(video_path2, arcname=new_name2)
            # Copy to folder if needed
            if to_folder:
                shutil.copy2(video_path2, os.path.join(folder_path, new_name2))
            added_videos2.append(episode_label)
        else:
            missing_videos2.append(episode_label)
    # Close the zip file if we created one
    if to_zip:
        zip_obj.close()
    # Print results
    print(f"Pack videos completed:")
    if to_zip:
        print(f"- Created zip file: {zip_file_path}")
    if to_folder:
        print(f"- Created folder: {folder_path}")
    print(f"Added {len(added_videos1)} videos from path1 and {len(added_videos2)} videos from path2.")
    if missing_videos1:
        print(f"Warning: Could not find {len(missing_videos1)} videos in path1:")
        print(missing_videos1)
    if missing_videos2:
        print(f"Warning: Could not find {len(missing_videos2)} videos in path2:")
        print(missing_videos2)
    # Add statistics to result
    result_paths['stats'] = {
        'added_path1': len(added_videos1),
        'added_path2': len(added_videos2),
        'missing_path1': len(missing_videos1),
        'missing_path2': len(missing_videos2)
    }
    return result_paths

# assume results2 have equal or less entries than results1

# folder1 = 'ext_dump/multy_dump/apr30_hm3d_Ours_ObjectOnly_ExploitOnly_ObservedOnly_stair_StairSpawn_EdgeGoal/objectnav-dino'
# folder2 = 'dump/object_exploitation_030/objectnav-dino'

# folder1 = '../ext_dump/object_exploitation_fix/objectnav-dino'
# folder2 = '../ext_dump/object_region_exploitation_prediction/objectnav-dino'

folder1 = '../ext_dump/object_exploitation_fix/objectnav-dino'
folder2 = '../ext_dump/new_region_caption_text/objectnav-dino'

log_path1 = f'{folder1}/logs/'
log_path2 = f'{folder2}/logs/'
vid_path1 = f'{folder1}/episodes_video'
vid_path2 = f'{folder2}/episodes_video'

In [14]:
# compare two sets of results

results1 = load_all_info(log_path1)
results2 = load_all_info(log_path2)
episode_labels1 = [x['episode'] for x in results1]
episode_labels2 = [x['episode'] for x in results2]

print(f"Number of episodes in {log_path1}: {len(episode_labels1)}")
print(f"Number of episodes in {log_path2}: {len(episode_labels2)}")

better1 = 0
better2 = 0

better_episode_labels1 = []
better_episode_labels2 = []
diff_episode_labels = []

for i2, episode_label2 in enumerate(episode_labels2):
    for i1, episode_label1 in enumerate(episode_labels1):
        if episode_label2 == episode_label1:
            # analyze the pair results
            result1 = results1[i1]
            result2 = results2[i2]
            if result1['habitat_success'] != result2['habitat_success']:
                diff_episode_labels.append(episode_label1)
            if result1['habitat_success'] > result2['habitat_success']:
                better1 += 1
                better_episode_labels1.append(episode_label1)
            elif result1['habitat_success'] < result2['habitat_success']:
                better2 += 1
                better_episode_labels2.append(episode_label1)
print(f"Better in {log_path1}: {better1}")
print(f"Better in {log_path2}: {better2}")
print(f"Better in {log_path1}:\nepisode_labels={better_episode_labels1}")
print(f"Better in {log_path2}:\nepisode_labels={better_episode_labels2}")
print(f"Different results:\nepisode_labels={diff_episode_labels}")

online_vid_path1 = 'http://localhost:8820/apr15_baseline_gt_perception_stair/objectnav-dino/episodes_video/'
online_vid_path2 = 'http://localhost:8820/apr21_baseline_gt_perception_stair/objectnav-dino/episodes_video/'
# print video paths for analyzing the differences
print(f"Better in {log_path1}:\n")
for episode_label in better_episode_labels1:
    vid_name = f'eps_{episode_label}_vis.mp4'
    print(os.path.abspath(os.path.join(vid_path1, vid_name)))
    print(os.path.abspath(os.path.join(vid_path2, vid_name)))
    # print(os.path.join(online_vid_path1, vid_name))
    # print(os.path.join(online_vid_path2, vid_name))
    print('')

print(f"Better in {log_path2}:\n")
for episode_label in better_episode_labels2:
    vid_name = f'eps_{episode_label}_vis.mp4'
    print(os.path.abspath(os.path.join(vid_path1, vid_name)))
    print(os.path.abspath(os.path.join(vid_path2, vid_name)))
    # print(os.path.join(online_vid_path1, vid_name))
    # print(os.path.join(online_vid_path2, vid_name))
    print('')

# analyze upstairs/downstairs
upstairs_success = []
upstairs_failure = []
downstairs_success = []
downstairs_failure = []
switched_count = 0
switched_success_count = 0
for episode in results2:
    if episode['switch_upstair_count']+episode['switch_downstair_count']>0:
        switched_count += 1
        if episode['habitat_success']:
            switched_success_count += 1
            if episode['switch_upstair_count'] > 0:
                upstairs_success.append(episode['episode'])
            elif episode['switch_downstair_count'] > 0:
                downstairs_success.append(episode['episode'])
        else:
            if episode['switch_upstair_count'] > 0:
                upstairs_failure.append(episode['episode'])
            elif episode['switch_downstair_count'] > 0:
                downstairs_failure.append(episode['episode'])
if switched_success_count > 0:
    print(f"Switched floor SR: {switched_count}/{switched_success_count} = {switched_count/switched_success_count:.2f}")
    print(f"Upstairs success: {len(upstairs_success)}\nepisode_labels={upstairs_success}")
    print(f"Upstairs failure: {len(upstairs_failure)}\nepisode_labels={upstairs_failure}")
    print(f"Downstairs success: {len(downstairs_success)}\nepisode_labels={downstairs_success}")
    print(f"Downstairs failure: {len(downstairs_failure)}\nepisode_labels={downstairs_failure}")


Loading ../ext_dump/object_exploitation_fix/objectnav-dino/logs/all_info.log
Loading ../ext_dump/new_region_caption_text/objectnav-dino/logs/all_info.log
Number of episodes in ../ext_dump/object_exploitation_fix/objectnav-dino/logs/: 400
Number of episodes in ../ext_dump/new_region_caption_text/objectnav-dino/logs/: 81
Better in ../ext_dump/object_exploitation_fix/objectnav-dino/logs/: 9
Better in ../ext_dump/new_region_caption_text/objectnav-dino/logs/: 6
Better in ../ext_dump/object_exploitation_fix/objectnav-dino/logs/:
episode_labels=['6s7QHgap2fW_94', 'Dd4bFSTQ8gi_93', 'bxsVRursffK_93', '5cdEh9F2hJL_72', 'cvZr5TUy5C5_3', 'DYehNKdT76V_68', 'Nfvxx8J5NCo_84', 'svBbv1Pavdk_65', 'XB4GS9ShBRE_98']
Better in ../ext_dump/new_region_caption_text/objectnav-dino/logs/:
episode_labels=['4ok3usBNeis_72', 'TEEsavR23oF_11', 'Nfvxx8J5NCo_78', 'q3zU7Yy5E5s_13', 'wcojb4TFT35_72', 'ziup5kvtCCR_70']
Different results:
episode_labels=['6s7QHgap2fW_94', '4ok3usBNeis_72', 'TEEsavR23oF_11', 'Dd4bFSTQ8gi_

In [ ]:
# 1 dtg>0.2
# 1 stop criterion
wcojb4TFT35_46_vis.mp4

In [ ]:
import constants

In [ ]:
# pack videos into zip according to failure case
output_path = '/home/junzhewu/Projects/sg-vln/dump/diff'
pack_videos(output_path, 'now_worse', vid_path1, vid_path2, better_episode_labels1, to_zip=True, to_folder=True)
pack_videos(output_path, 'now_better', vid_path1, vid_path2, better_episode_labels2, to_zip=True, to_folder=True)
pack_videos(output_path, 'upstairs_success', vid_path1, vid_path2, upstairs_success, to_zip=True, to_folder=True)
pack_videos(output_path, 'upstairs_failure', vid_path1, vid_path2, upstairs_failure, to_zip=True, to_folder=True)
pack_videos(output_path, 'downstairs_success', vid_path1, vid_path2, downstairs_success, to_zip=True, to_folder=True)
pack_videos(output_path, 'downstairs_failure', vid_path1, vid_path2, downstairs_failure, to_zip=True, to_folder=True)

In [29]:
# calculate mean spl
import numpy as np
results = load_all_info(log_path1)
spl_list = []
soft_spl_list = []
num_success = 0
for result in results:
    if result['habitat_success'] == 1:
        num_success += 1
    soft_spl_list.append(result['soft_spl'])
    if np.isnan(result['spl']):
        spl_list.append(0)
    else:
        spl_list.append(result['spl'])
print("spl:", np.mean(spl_list))
print("soft spl:", np.mean(soft_spl_list))
print("success rate:", num_success / len(results), num_success)

Loading ../ext_dump/object_exploitation_fix/objectnav-dino/logs/all_info.log
spl: 0.27322853118462964
soft spl: 0.30811769102013253
success rate: 0.625 250


In [32]:
# show results for specific episodes
episode_labels = [x['episode'] for x in load_all_info(log_path2)]
results = load_all_info(log_path1)
count = 0
success_count = 0
spl_list = []
softspl_list = []
for result in results:
    if result['episode'] in episode_labels:
        # print(f"Episode: {result['episode']}")
        # print(f"habitat_success: {result['habitat_success']}")
        # print(f"upstair_flag: {result['upstair_flag']}")
        # print(f"downstair_flag: {result['downstair_flag']}")
        # print(f"spl: {result['spl']}")
        # print(f"distance_to_goal: {result['distance_to_goal']}")
        # print("\n")
        count += 1
        if result['habitat_success']:
            success_count += 1
        if np.isnan(result['spl']):
            spl_list.append(0)
        else:
            spl_list.append(result['spl'])
        if np.isnan(result['soft_spl']):
            softspl_list.append(0)
        else:
            softspl_list.append(result['soft_spl'])
print(f"Success count: {success_count}/{count}")
print(f"Success rate: {success_count/count}")
print("spl:", np.mean(spl_list))
print("softspl:", np.mean(softspl_list))
print(f"Found episodes: {count}")

Loading ../ext_dump/new_region_caption_text/objectnav-dino/logs/all_info.log
Loading ../ext_dump/object_exploitation_fix/objectnav-dino/logs/all_info.log
Success count: 57/80
Success rate: 0.7125
spl: 0.30719508185540706
softspl: 0.3508916379708975
Found episodes: 80


In [ ]:
# show results for specific episodes
episode_labels = ["bxsVRursffK_43", "bxsVRursffK_56", "Dd4bFSTQ8gi_63", "Dd4bFSTQ8gi_66", "qyAac8rV8Zk_4", "TEEsavR23oF_47", "TEEsavR23oF_52", "wcojb4TFT35_17", "wcojb4TFT35_35", "zt1RVoi7PcG_84", "zt1RVoi7PcG_94"]
results = load_all_info(log_path2)
count = 0
success_count = 0
for result in results:
    if result['episode'] in episode_labels:
        print(f"Episode: {result['episode']}")
        print(f"habitat_success: {result['habitat_success']}")
        # print(f"upstair_flag: {result['upstair_flag']}")
        # print(f"downstair_flag: {result['downstair_flag']}")
        # print(f"spl: {result['spl']}")
        # print(f"distance_to_goal: {result['distance_to_goal']}")
        print("\n")
        count += 1
        if result['habitat_success']:
            success_count += 1
print(f"Success count: {success_count}/{count}")
print(f"Success rate: {success_count/count}")
print(f"Found episodes: {count}")

In [ ]:
# found goal but failed
log_path = './dump/new_baseline_apr2/objectnav-dino/logs/'
vid_path = './dump/new_baseline_apr2/objectnav-dino/episodes_video/'
results = load_all_info(log_path2)
count_fail = 0
fail_episode_labels = []
for result in results:
    if result['habitat_success'] == 0 and result['distance_to_goal']<0.2:
        print(f"Episode {result['episode']}: ({result['habitat_success']}, {result['distance_to_goal']})")
        vid_name = f'eps_{episode_label}_vis.mp4'
        print(os.path.join(vid_path, vid_name))
        count_fail += 1
        fail_episode_labels.append(result['episode'])
print(f"Number of episodes with found goal but failed: {count_fail}")
print(f"episode_labels={fail_episode_labels}")

In [ ]:
# # remove results (comment out to avoid accidental deletion)
# import os
# episode_labels = ["bxsVRursffK_43", "bxsVRursffK_56", "Dd4bFSTQ8gi_63", "Dd4bFSTQ8gi_66", "qyAac8rV8Zk_4", "TEEsavR23oF_47", "TEEsavR23oF_52", "wcojb4TFT35_17", "wcojb4TFT35_35", "zt1RVoi7PcG_84", "zt1RVoi7PcG_94"]

# results = load_all_info(log_path2)
# for episode_label in episode_labels:
#     # remove video
#     video_path = f'{vid_path2}eps_{episode_label}_vis.mp4'
#     if os.path.exists(video_path):
#         print(f"Removing {video_path}")
#         os.system(f"rm {video_path}")
#     else:
#         print(f"{video_path} does not exist")
#     # remove result
#     for i, result in enumerate(results):
#         if result['episode'] == episode_label:
#             print(f"Removing {log_path2}all_info.log entry {i}")
#             print(f"    Removing {results[i]}")
#             results.pop(i)
#             break
# save_all_info(log_path2, results)
    

In [ ]:
# Example usage of the pack_videos_into_zip function

# For videos where approach 1 performed better
zip_path = './dump/analysis/'
videos_path = './dump/new_baseline_apr2/objectnav-dino/episodes_video/'

# Pack videos where approach 1 was better
pack_videos(
    zip_path=zip_path,
    zip_filename='approach1_better_videos',
    videos_path=videos_path,
    episode_labels=better_episode_labels1
)

# Pack videos where approach 2 was better
pack_videos(
    zip_path=zip_path,
    zip_filename='approach2_better_videos',
    videos_path=videos_path,
    episode_labels=better_episode_labels2
)

In [ ]:
# load all episodes
episode_data = load_all_episodes(env)  # load all episodes
env = load_env()
episode_labels = get_episode_labels(env)
print(f"Episode labels: {episode_labels}")
env.close()

In [ ]:
#randomly pick n_sample episodes for each scene
scene_dict = {}
n_sample_each_scene = 1
for episode_label in episode_labels:
    scene_id = episode_label.split('_')[0]
    scene_dict.setdefault(scene_id, []).append(episode_label)

import random
random.seed(3)
sample_episode_labels = []
for k, v in scene_dict.items():
    sample_episode_labels += random.sample(v, n_sample_each_scene)
print(f"Sample episode labels: {len(sample_episode_labels)}")
print(sample_episode_labels)

log_path1 = 'dump/fmm/objectnav-dino/logs/'
results1 = load_all_info(log_path1)
sample_results1 = []
count_success = 0
for i, result in enumerate(results1):
    if result['episode'] in sample_episode_labels:
        sample_results1.append(result)
        if result['habitat_success']:
            count_success += 1
print(f"Sample results in {log_path1}: {len(sample_results1)}")
print(f"Sample success in {log_path1}: {count_success}")
print(f"Sample success rate in {log_path1}: {count_success / len(sample_results1)}")


In [ ]:
#randomly pick n_sample episode labels
n_sample = 400
import random
random.seed(0)


results1 = load_all_info(log_path1)
results2 = load_all_info(log_path2)
episode_labels1 = [x['episode'] for x in results1]


folder1 = 'dump/fmm/objectnav-dino'
episode_labels = 
sample_episode_labels = random.sample(episode_labels, n_sample)
print(f"Sample episode labels: {len(sample_episode_labels)}")
print(sample_episode_labels)
log_path1 = 'dump/fmm/objectnav-dino/logs/'
results1 = load_all_info(log_path1)
sample_results1 = []
count_success = 0
for i, result in enumerate(results1):
    if result['episode'] in sample_episode_labels:
        sample_results1.append(result)
        if result['habitat_success']:
            count_success += 1
print(f"Sample results in {log_path1}: {len(sample_results1)}")
print(f"Sample success in {log_path1}: {count_success}")
print(f"Sample success rate in {log_path1}: {count_success / len(sample_results1)}")

In [ ]:
#randomly pick n_sample episode labels from sample 400
from constants import episode_labels_table
n_sample = 101
import random
random.seed(15)


episode_labels = episode_labels_table['sample400']
sample_episode_labels = random.sample(episode_labels, n_sample)
print(f"Sample episode labels: {len(sample_episode_labels)}")
print(sample_episode_labels)
log_path1 = 'dump/fmm/objectnav-dino/logs/'
results1 = load_all_info(log_path1)
sample_results1 = []
count_success = 0
for i, result in enumerate(results1):
    if result['episode'] in sample_episode_labels:
        sample_results1.append(result)
        if result['habitat_success']:
            count_success += 1
print(f"Sample results in {log_path1}: {len(sample_results1)}")
print(f"Sample success in {log_path1}: {count_success}")
print(f"Sample success rate in {log_path1}: {count_success / len(sample_results1)}")

In [ ]:
# icra_late_breaking
episode_labels = ['4ok3usBNeis_69', '6s7QHgap2fW_82', '6s7QHgap2fW_28', 'q3zU7Yy5E5s_74', 'bxsVRursffK_49', 'zt1RVoi7PcG_3', 'Dd4bFSTQ8gi_63', 'zt1RVoi7PcG_29', 'QaLdnwvtxbs_50', 'svBbv1Pavdk_65', 'wcojb4TFT35_70', 'qyAac8rV8Zk_85', 'p53SfW6mjZe_87', 'mv2HUxq3B53_42', 'TEEsavR23oF_60', 'qyAac8rV8Zk_23', '6s7QHgap2fW_44', 'Nfvxx8J5NCo_42', 'svBbv1Pavdk_90', '6s7QHgap2fW_15', 'q3zU7Yy5E5s_84', 'TEEsavR23oF_43', 'svBbv1Pavdk_43', '5cdEh9F2hJL_26', 'wcojb4TFT35_92', 'mv2HUxq3B53_41', 'cvZr5TUy5C5_73', 'Nfvxx8J5NCo_66', 'QaLdnwvtxbs_62', 'cvZr5TUy5C5_2', 'TEEsavR23oF_5', 'Nfvxx8J5NCo_68', 'QaLdnwvtxbs_21', 'TEEsavR23oF_11', '5cdEh9F2hJL_82', 'mL8ThkuaVTM_59', 'svBbv1Pavdk_63', 'TEEsavR23oF_91', 'DYehNKdT76V_18', 'bxsVRursffK_29', 'QaLdnwvtxbs_96', 'svBbv1Pavdk_32', '4ok3usBNeis_25', 'p53SfW6mjZe_32', '5cdEh9F2hJL_75', 'wcojb4TFT35_41', 'TEEsavR23oF_71', 'ziup5kvtCCR_20', 'QaLdnwvtxbs_77', 'p53SfW6mjZe_42', 'ziup5kvtCCR_14', 'DYehNKdT76V_94', 'bxsVRursffK_67', 'XB4GS9ShBRE_72', 'mv2HUxq3B53_10', 'Nfvxx8J5NCo_62', 'mv2HUxq3B53_12', '5cdEh9F2hJL_65', 'Nfvxx8J5NCo_13', 'XB4GS9ShBRE_53', 'mv2HUxq3B53_52', 'wcojb4TFT35_94', 'mL8ThkuaVTM_3', 'zt1RVoi7PcG_4', 'bxsVRursffK_4', 'DYehNKdT76V_95', 'ziup5kvtCCR_64', '5cdEh9F2hJL_61', '6s7QHgap2fW_48', 'ziup5kvtCCR_22', 'svBbv1Pavdk_55', 'cvZr5TUy5C5_3', 'Nfvxx8J5NCo_78', 'zt1RVoi7PcG_0', 'cvZr5TUy5C5_18', 'zt1RVoi7PcG_84', 'cvZr5TUy5C5_90', 'mv2HUxq3B53_0', 'XB4GS9ShBRE_20', 'q3zU7Yy5E5s_80', '5cdEh9F2hJL_50', 'DYehNKdT76V_0', 'qyAac8rV8Zk_30', 'DYehNKdT76V_44', 'bxsVRursffK_64', 'wcojb4TFT35_15', 'qyAac8rV8Zk_91', 'Dd4bFSTQ8gi_1', 'mL8ThkuaVTM_23', 'DYehNKdT76V_56', 'cvZr5TUy5C5_61', 'DYehNKdT76V_74', 'mv2HUxq3B53_90', 'bxsVRursffK_52', 'TEEsavR23oF_40', 'Nfvxx8J5NCo_40', 'zt1RVoi7PcG_101', 'ziup5kvtCCR_77', 'zt1RVoi7PcG_1', '6s7QHgap2fW_68', 'QaLdnwvtxbs_12', 'wcojb4TFT35_8', 'XB4GS9ShBRE_31', 'bxsVRursffK_19', 'q3zU7Yy5E5s_23', 'cvZr5TUy5C5_81', 'mv2HUxq3B53_55', '5cdEh9F2hJL_21', 'mL8ThkuaVTM_78', 'ziup5kvtCCR_49', 'XB4GS9ShBRE_55', 'qyAac8rV8Zk_37', 'mL8ThkuaVTM_19', '4ok3usBNeis_95', 'wcojb4TFT35_56', 'TEEsavR23oF_47', 'wcojb4TFT35_35', '4ok3usBNeis_62', 'QaLdnwvtxbs_83', 'bxsVRursffK_98', '6s7QHgap2fW_95', 'XB4GS9ShBRE_62', 'QaLdnwvtxbs_90', 'mv2HUxq3B53_28', 'wcojb4TFT35_48', 'bxsVRursffK_50', 'svBbv1Pavdk_28', 'svBbv1Pavdk_40', 'qyAac8rV8Zk_56', 'TEEsavR23oF_1', '4ok3usBNeis_48', '5cdEh9F2hJL_84', '6s7QHgap2fW_59', 'mL8ThkuaVTM_20', 'qyAac8rV8Zk_86', 'zt1RVoi7PcG_42', 'ziup5kvtCCR_28', 'bxsVRursffK_69', 'q3zU7Yy5E5s_3', 'Dd4bFSTQ8gi_93', 'qyAac8rV8Zk_2', 'DYehNKdT76V_66', 'bxsVRursffK_17', 'qyAac8rV8Zk_45', '5cdEh9F2hJL_37', '6s7QHgap2fW_73', 'qyAac8rV8Zk_41', '5cdEh9F2hJL_72', 'TEEsavR23oF_50', 'zt1RVoi7PcG_26', 'qyAac8rV8Zk_12', 'mL8ThkuaVTM_91', 'TEEsavR23oF_78', 'zt1RVoi7PcG_24', 'mv2HUxq3B53_25', '4ok3usBNeis_6', 'DYehNKdT76V_68', 'bxsVRursffK_56', '4ok3usBNeis_58', '5cdEh9F2hJL_36', 'bxsVRursffK_93', 'Dd4bFSTQ8gi_69', 'qyAac8rV8Zk_0', 'p53SfW6mjZe_90', '6s7QHgap2fW_45', 'svBbv1Pavdk_88', 'DYehNKdT76V_54', '6s7QHgap2fW_80', 'XB4GS9ShBRE_73', 'DYehNKdT76V_6', 'mv2HUxq3B53_38', 'Nfvxx8J5NCo_1', 'Nfvxx8J5NCo_90', 'zt1RVoi7PcG_78', 'Nfvxx8J5NCo_75', 'QaLdnwvtxbs_38', '4ok3usBNeis_71', 'p53SfW6mjZe_44', 'mv2HUxq3B53_98', 'mv2HUxq3B53_62', 'qyAac8rV8Zk_58', 'QaLdnwvtxbs_53', 'TEEsavR23oF_46', 'zt1RVoi7PcG_116', 'XB4GS9ShBRE_9', 'q3zU7Yy5E5s_43', 'bxsVRursffK_18', 'mL8ThkuaVTM_76', 'wcojb4TFT35_72', 'qyAac8rV8Zk_11', 'Dd4bFSTQ8gi_92', '5cdEh9F2hJL_76', 'bxsVRursffK_7', 'TEEsavR23oF_38', 'mv2HUxq3B53_67', 'qyAac8rV8Zk_33', 'svBbv1Pavdk_12', 'mv2HUxq3B53_26', 'Dd4bFSTQ8gi_95', 'bxsVRursffK_72', '5cdEh9F2hJL_80', 'wcojb4TFT35_60', 'svBbv1Pavdk_38', 'mv2HUxq3B53_16', 'QaLdnwvtxbs_91', 'wcojb4TFT35_1', 'Nfvxx8J5NCo_56', 'p53SfW6mjZe_59', 'p53SfW6mjZe_8', 'XB4GS9ShBRE_88', '6s7QHgap2fW_17', '6s7QHgap2fW_84', 'ziup5kvtCCR_21', 'TEEsavR23oF_96', 'XB4GS9ShBRE_17', 'wcojb4TFT35_26', 'mL8ThkuaVTM_57', 'DYehNKdT76V_85', 'Nfvxx8J5NCo_26', 'XB4GS9ShBRE_11', 'QaLdnwvtxbs_19', 'ziup5kvtCCR_31', 'mL8ThkuaVTM_86', 'qyAac8rV8Zk_19', 'qyAac8rV8Zk_4', 'svBbv1Pavdk_56', '5cdEh9F2hJL_93', 'zt1RVoi7PcG_113', '4ok3usBNeis_63', 'qyAac8rV8Zk_34', 'mL8ThkuaVTM_60', 'zt1RVoi7PcG_52', '5cdEh9F2hJL_40', '5cdEh9F2hJL_34', 'TEEsavR23oF_3', 'DYehNKdT76V_58', 'wcojb4TFT35_73', 'cvZr5TUy5C5_59', 'ziup5kvtCCR_30', 'ziup5kvtCCR_44', '5cdEh9F2hJL_23', 'cvZr5TUy5C5_85', '4ok3usBNeis_72', 'qyAac8rV8Zk_81', 'Dd4bFSTQ8gi_58', 'QaLdnwvtxbs_10', 'zt1RVoi7PcG_94', 'q3zU7Yy5E5s_54', 'q3zU7Yy5E5s_60', 'Dd4bFSTQ8gi_66', 'XB4GS9ShBRE_98', 'TEEsavR23oF_82', 'ziup5kvtCCR_70', '5cdEh9F2hJL_6', 'svBbv1Pavdk_44', 'wcojb4TFT35_32', 'cvZr5TUy5C5_88', 'XB4GS9ShBRE_7', 'Nfvxx8J5NCo_7', 'p53SfW6mjZe_88', 'mv2HUxq3B53_23', 'cvZr5TUy5C5_20', '6s7QHgap2fW_53', 'Dd4bFSTQ8gi_47', 'wcojb4TFT35_9', 'p53SfW6mjZe_43', 'q3zU7Yy5E5s_22', '5cdEh9F2hJL_0', 'ziup5kvtCCR_5', 'mL8ThkuaVTM_41', 'q3zU7Yy5E5s_41', '4ok3usBNeis_7', 'Dd4bFSTQ8gi_5', 'qyAac8rV8Zk_16', 'mv2HUxq3B53_40', 'ziup5kvtCCR_80', '5cdEh9F2hJL_81', '6s7QHgap2fW_98', 'p53SfW6mjZe_5', 'bxsVRursffK_30', 'Nfvxx8J5NCo_6', 'wcojb4TFT35_17', '6s7QHgap2fW_16', 'mL8ThkuaVTM_24', 'qyAac8rV8Zk_87', 'QaLdnwvtxbs_76', 'QaLdnwvtxbs_78', 'QaLdnwvtxbs_68', '4ok3usBNeis_18', 'mL8ThkuaVTM_28', 'TEEsavR23oF_52', 'QaLdnwvtxbs_26', 'svBbv1Pavdk_20', 'DYehNKdT76V_2', 'TEEsavR23oF_2', '5cdEh9F2hJL_98', 'svBbv1Pavdk_5', '6s7QHgap2fW_34', 'DYehNKdT76V_13', 'DYehNKdT76V_69', 'XB4GS9ShBRE_4', 'mL8ThkuaVTM_74', 'mv2HUxq3B53_17', 'svBbv1Pavdk_6', '4ok3usBNeis_87', 'mL8ThkuaVTM_39', 'ziup5kvtCCR_53', '4ok3usBNeis_56', 'Nfvxx8J5NCo_5', 'svBbv1Pavdk_41', 'DYehNKdT76V_63', '4ok3usBNeis_43', 'wcojb4TFT35_75', 'TEEsavR23oF_57', 'ziup5kvtCCR_55', 'XB4GS9ShBRE_23', 'svBbv1Pavdk_86', 'ziup5kvtCCR_76', 'DYehNKdT76V_98', 'zt1RVoi7PcG_75', 'ziup5kvtCCR_36', 'wcojb4TFT35_63', 'XB4GS9ShBRE_69', 'Dd4bFSTQ8gi_14', 'cvZr5TUy5C5_22', 'mL8ThkuaVTM_87', 'Dd4bFSTQ8gi_75', 'Nfvxx8J5NCo_10', 'TEEsavR23oF_64', 'p53SfW6mjZe_48', 'TEEsavR23oF_88', 'TEEsavR23oF_54', 'wcojb4TFT35_22', 'Dd4bFSTQ8gi_51', 'bxsVRursffK_35', 'mv2HUxq3B53_43', '4ok3usBNeis_37', 'svBbv1Pavdk_9', 'XB4GS9ShBRE_83', '4ok3usBNeis_12', 'mL8ThkuaVTM_0', 'Dd4bFSTQ8gi_29', 'wcojb4TFT35_46', 'mL8ThkuaVTM_21', 'mL8ThkuaVTM_82', 'bxsVRursffK_26', 'Dd4bFSTQ8gi_60', 'bxsVRursffK_43', 'wcojb4TFT35_52', 'Dd4bFSTQ8gi_56', 'p53SfW6mjZe_93', 'mv2HUxq3B53_64', 'Nfvxx8J5NCo_98', 'qyAac8rV8Zk_22', 'mv2HUxq3B53_96', '6s7QHgap2fW_94', 'ziup5kvtCCR_72', 'zt1RVoi7PcG_69', 'p53SfW6mjZe_37', 'p53SfW6mjZe_66', 'zt1RVoi7PcG_103', 'q3zU7Yy5E5s_58', 'DYehNKdT76V_27', 'mL8ThkuaVTM_56', 'qyAac8rV8Zk_66', '6s7QHgap2fW_37', 'zt1RVoi7PcG_63', 'wcojb4TFT35_95', 'q3zU7Yy5E5s_13', 'QaLdnwvtxbs_92', 'cvZr5TUy5C5_65', 'ziup5kvtCCR_32', 'DYehNKdT76V_62', 'mL8ThkuaVTM_51', 'Nfvxx8J5NCo_84', 'p53SfW6mjZe_29', 'QaLdnwvtxbs_23', 'cvZr5TUy5C5_97', 'wcojb4TFT35_88', 'DYehNKdT76V_97', 'mv2HUxq3B53_88', '4ok3usBNeis_16', 'p53SfW6mjZe_56', 'ziup5kvtCCR_0', 'Dd4bFSTQ8gi_15', 'cvZr5TUy5C5_26', 'mL8ThkuaVTM_63', 'qyAac8rV8Zk_7', 'DYehNKdT76V_31', '6s7QHgap2fW_62', 'q3zU7Yy5E5s_51', 'bxsVRursffK_74', 'ziup5kvtCCR_94', 'bxsVRursffK_76', 'bxsVRursffK_25', 'TEEsavR23oF_61', 'ziup5kvtCCR_88', '6s7QHgap2fW_61', 'svBbv1Pavdk_62', 'ziup5kvtCCR_23']
